In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.9 MB/s 
     |████████████████████████████████| 596 kB 17.2 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 31.9 MB/s 
     |████████████████████████████████| 6.5 MB 41.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
train_data = pd.read_csv('raw_data/train.csv')
X_test = pd.read_csv('raw_data/test.csv')

In [21]:
X_tv = train_data['text']
y_tv = train_data['target']

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size = .3, random_state = 42)

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
X_train.head()

1186    Ashes 2015: AustraliaÛªs collapse at Trent Br...
4071    GREAT MICHIGAN TECHNIQUE CAMP\nB1G THANKS TO @...
5461    CNN: Tennessee movie theater shooting suspect ...
5787    Still rioting in a couple of hours left until ...
7445    Crack in the path where I wiped out this morni...
Name: text, dtype: object

In [10]:
X_train[7445]

'Crack in the path where I wiped out this morning during beach run. Surface wounds on left elbow and right knee. http://t.co/yaqRSximph'

In [11]:
X_train = X_train.apply(lambda x : np.array(tokenizer.encode(x)))

In [12]:
X_val = X_val.apply(lambda x : np.array(tokenizer.encode(x)))

In [22]:
X_tv = pad_sequences(X_tv.apply(lambda x : tokenizer.encode(x)), padding = 'post')

In [23]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [15]:
es = EarlyStopping(patience = 2, restore_best_weights = True)

In [24]:
history = model.fit(X_tv, y_tv, callbacks = [es], epochs = 100, validation_split = .2, batch_size = 32)

Epoch 1/100
191/191 [==============================] - 263s 1s/step - loss: 8.7174 - accuracy: 0.4251 - val_loss: 8.1503 - val_accuracy: 0.4655
Epoch 2/100
191/191 [==============================] - 200s 1s/step - loss: 8.8340 - accuracy: 0.4207 - val_loss: 8.1503 - val_accuracy: 0.4655
Epoch 3/100
191/191 [==============================] - 200s 1s/step - loss: 8.8340 - accuracy: 0.4207 - val_loss: 8.1503 - val_accuracy: 0.4655


In [26]:
X_test = pad_sequences(X_test['text'].apply(lambda x : tokenizer.encode(x)), padding = 'post')

In [28]:
predictions = model.predict(X_test)